# Data Cleaning

This part will be doing some data cleaning.

In [1]:
#pip install ipynb

In [2]:
#pip install joblib

In [3]:
#pip install mlxtend

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib # If you get error change container or open terminal and run the below comment
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [5]:
raw = pd.read_csv('/dsa/groups/capstonesp2022/on-campus/group_1/data/combined_dataset.csv')

In [6]:
df=raw.iloc[:,1:] # Removing first column ['Unnamed: 0']

### Target HBP

Creating target column to identify high blood pressure.

In [7]:
# get the mean of sy(systolic blood pressure) and di ( diastolic blood pressure) from three measurements
df['BPX_SY'] = (df['bpxsy1_systolic__blood_pres_1st_rdg_mm_hg']+df['bpxsy2_systolic__blood_pres_2nd_rdg_mm_hg']+df['bpxsy3_systolic__blood_pres_3rd_rdg_mm_hg'])/3
df['BPX_DI'] = (df['bpxdi1_diastolic__blood_pres_1st_rdg_mm_hg']+df['bpxdi2_diastolic__blood_pres_2nd_rdg_mm_hg']+df['bpxdi3_diastolic__blood_pres_3rd_rdg_mm_hg'])/3

From our research, we learned that if a person's systolic blood pressure is greater or equal to 130, we can say that they have high blood pressure. Also, if a person's diastolic blood pressure is greater or equal to 80, we can say that they have high blood pressure.

In [8]:
df['BPX']=np.where(((df['BPX_SY'] >=130) | (df['BPX_DI'] >=80)) & (df['BPX_SY'] > df['BPX_DI']),1,0)

Using five different columns we will create our final target column.

In [9]:
# combine three condition data
df['HBP'] = np.where((df['BPX'] ==1) | 
                     (df['diq175h_high_blood_pressure'] == 17) | 
                     (df['bpq050a_now_taking_prescribed_medicine_for_hbp'] ==1) |
                     (df['bpq020_ever_told_you_had_high_blood_pressure'] ==1) |
                     (df['bpq030_told_had_high_blood_pressure'] ==1) ,1,0)

In [10]:
df['HBP'].value_counts()

0    32805
1    14847
Name: HBP, dtype: int64

In [11]:
df.shape

(47652, 53)

# Data cleaning

We remove people who are under 20 years old because most of the information that we are looking at is empty. For adults, 80 years and older was determined to be a disclosure risk so CDC maps people who are older than 80 to 80. For example, if the patient is 85 years old, in the dataset this person's age will be 80. That is why we want to exclude people who are 80 or older. In the end, our population age distribution will be between 20 and 79.

In [12]:
df = df[(df['ridageyr_age_in_years_at_screening']>=20)]
df = df[(df['ridageyr_age_in_years_at_screening']<80)]
df.shape

(25926, 53)

In [13]:
df['HBP'].value_counts()

1    13027
0    12899
Name: HBP, dtype: int64

In our case study, we were using only data from 2017-2018 and we when pull the previous years' data, we saw that some survey questions are changed. For example, in the alcohol dataset between 2017 and 2018, our alcohol-related column values are mapped to specific categories such as "Every day", "Nearly every day" etc. However, when we look at the previous years, they gave an estimated number for one column and another column for the unit. The first column shows how many times they drink alcohol, and the second column shows the unit (per week, per month, and per year) of the column. To create a single logic, we need to convert previous years' data to the current format.

In [14]:
df['flag'] = 999

In [15]:
# Every day
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 1) & # per week \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 7 )
       ,'flag'] = 1 

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) & # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 30)
       ,'flag'] = 1 

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 365 )
       ,'flag'] = 1

In [16]:
# Nearly every day
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 1) & # per week \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 7 ) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 5 )
       ,'flag'] = 2 

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) & # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 30 ) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 17)
       ,'flag'] = 2 

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 365 ) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 209) \
       ,'flag'] = 2

In [17]:
# 3 to 4 times a week
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 1) & # per week \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 5 ) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 3 )
       ,'flag'] = 3

In [18]:
# 2 times a week
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 1) & # per week \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 2 )
       ,'flag'] = 4

In [19]:
# Once a week
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 1) &  # per week \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 1 )
       ,'flag'] = 5

In [20]:
# Once a month
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) & # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 1 )
       ,'flag'] = 7 


df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 12 )
       ,'flag'] = 7

In [21]:
# 2 to 3 times a month
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) &  # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] > 1) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] <= 3 )
       ,'flag'] = 6

In [22]:
# 1 to 2 times in the last year
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] <= 2 )
       ,'flag'] = 10

In [23]:
# 3 to 6 times in the last year
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] > 2 ) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] <= 6 )
       ,'flag'] = 9 

In [24]:
# 7 to 11 times in the last year
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] > 6 ) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] <= 11 )
       ,'flag'] = 8 

In [25]:
#per year logic
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 156) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 209 )
       ,'flag'] = 3

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 104) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 156 )
       ,'flag'] = 4

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 52) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 104 )
       ,'flag'] = 5

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 24) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 52 )
       ,'flag'] = 6

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 3) & # per year \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] >= 12) & \
       (df['flag'] == 999 ) & \
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] < 24 )
       ,'flag'] = 7

In [26]:
#per year logic
df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) & # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) >= 156) & \
       (df['flag'] == 999 ) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) < 209 )
       ,'flag'] = 3

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) & # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) >= 104) & \
       (df['flag'] == 999 ) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) < 156 )
       ,'flag'] = 4

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) & # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) >= 52) & \
       (df['flag'] == 999 ) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) < 104 )
       ,'flag'] = 5

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) & # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) >= 24) & \
       (df['flag'] == 999 ) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) < 52 )
       ,'flag'] = 6

df.loc[(df['alq141u_days_drink_alcohol_per_wk_mo_yr'] == 2) & # per month \ 
       (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].notnull()) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) >= 12) & \
       (df['flag'] == 999 ) & \
       ((df['alq142_number_days_have_4_or_5_drinkspast_12_mos']*12) < 24 )
       ,'flag'] = 7

In [27]:
filter_flag = (df.flag!=999)
df.loc[filter_flag, 'alq142_number_days_have_4_or_5_drinkspast_12_mos'] = df['flag']

Alcohol column imputation.

In [28]:
# People who don't drink at all
df.loc[(df['alq111_ever_had_a_drink_of_any_kind_of_alcohol'] == 2) , 'alq142_number_days_have_4_or_5_drinkspast_12_mos'] = 11

# People who don't drink 4\5 
df.loc[(df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].isna()) & 
   (df['alq111_ever_had_a_drink_of_any_kind_of_alcohol'] == 1) & 
   (df['alq151_ever_have_4_5_or_more_drinks_every_day'] == 2) , 'alq142_number_days_have_4_or_5_drinkspast_12_mos'] = 99

# Don't know
df.loc[(df['alq142_number_days_have_4_or_5_drinkspast_12_mos'].isna()), 'alq142_number_days_have_4_or_5_drinkspast_12_mos'] = 99
df.loc[(df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 77), 'alq142_number_days_have_4_or_5_drinkspast_12_mos'] = 99
df.loc[(df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 777), 'alq142_number_days_have_4_or_5_drinkspast_12_mos'] = 99
df.loc[(df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 999), 'alq142_number_days_have_4_or_5_drinkspast_12_mos'] = 99
df.loc[(df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 11), 'alq142_number_days_have_4_or_5_drinkspast_12_mos'] = 0

df.loc[(df['alq111_ever_had_a_drink_of_any_kind_of_alcohol'].isnull()) , 'alq111_ever_had_a_drink_of_any_kind_of_alcohol'] = 9

# Making sure that we don't get any other invalid value
df = df[(df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 0) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 1) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 2) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 3) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 4) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 5) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 6) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 7) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 8) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 9) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 10) | \
        (df['alq142_number_days_have_4_or_5_drinkspast_12_mos'] == 99)]

Creating salt usage column

In [29]:
# 9 Means don't know 
df.loc[(df['dbd100_how_often_add_salt_to_food_at_table'] == 9) , 'dbd100_how_often_add_salt_to_food_at_table'] = 0
df.loc[(df['drqsprep_salt_used_in_preparation'] == 9) , 'drqsprep_salt_used_in_preparation'] = 0
df.loc[(df['dbd100_how_often_add_salt_to_food_at_table'].isnull()) , 'dbd100_how_often_add_salt_to_food_at_table'] = 0
df.loc[(df['drqsprep_salt_used_in_preparation'].isnull()) , 'drqsprep_salt_used_in_preparation'] = 0

df['salt_usage'] = None

# Categorizing salt usage
df.loc[(df['drqsprep_salt_used_in_preparation'] == 4)  & 
   (df['dbd100_how_often_add_salt_to_food_at_table'] == 3) , 'salt_usage'] = 1


df.loc[(df['drqsprep_salt_used_in_preparation'] == 3)  & 
   (df['dbd100_how_often_add_salt_to_food_at_table'] == 3) , 'salt_usage'] = 1


df.loc[(df['salt_usage'].isnull()) , 'salt_usage'] = 0

Creating BMI column.

In [30]:
df['BMI'] = round(((df['whd020_current_self'] * 0.45359237) / ((df['whd010_current_self'] * 2.54)**2)) * 10000,1)
df = df.drop(columns=[
    'whd020_current_self','whd010_current_self'])

Creating new column for Insulin.

In [31]:
df['insulin_usage'] = None
df.loc[(df['diq060u_unit_of_measure_month_year']==1) &  # Months
   (df['did060_how_long_taking_insulin'] < 3) , 'insulin_usage'] = '2 Month'

df.loc[(df['diq060u_unit_of_measure_month_year']==1) &  # Months
   (df['did060_how_long_taking_insulin'] >= 3) & \
   (df['did060_how_long_taking_insulin'] < 5)  , 'insulin_usage'] = '3-4 Months'

df.loc[(df['diq060u_unit_of_measure_month_year']==1) &  # Months
   (df['did060_how_long_taking_insulin'] >= 5), 'insulin_usage'] = '5-12 Months'

df.loc[(df['diq060u_unit_of_measure_month_year']==2), 'insulin_usage'] = 'More than one year' 

df.loc[(df['diq060u_unit_of_measure_month_year'].isnull()), 'diq060u_unit_of_measure_month_year'] = 0
df.loc[(df['did060_how_long_taking_insulin'].isnull()), 'did060_how_long_taking_insulin'] = 0
df.loc[(df['insulin_usage'].isnull()), 'insulin_usage'] = 'None'

In the race column, we have values 5 and 7 which are means Multi-Race. Since they have the same meaning we will use only one for the Multi-Race category.

In [32]:
df.loc[(df['ridreth3_racehispanic_origin_w_nh_asian'] == 7) , 'ridreth3_racehispanic_origin_w_nh_asian'] = 5

### 1. Dropping columns according to unique values

In [33]:
# Unique value cheking
for column in df.columns:
    if len(df[column].unique()) == 1:
        print('Single value in the column : ', column)
        print('\tValues : ', df[column].unique())
        df = df.drop(columns=[column])
    #elif len(df[column].unique()) == 2 and df[column].isnull().values.any(): # Useless columns 
        #print('Useless column : ', column)
        #print('\tValues : ', df[column].unique())
        #df = df.drop(columns=[column])

### 2. Remove missing columns that have missing values >61%

In [34]:
null=100*(df.isnull().sum())/(df.shape[0])
df_null=pd.DataFrame({'percentage':null})
df_high=df_null[df_null['percentage']>65]
display(df_high.sort_values(by=['percentage']))

,percentage
bpq030_told_had_high_blood_pressure,66.037312
bpq050a_now_taking_prescribed_medicine_for_hbp,70.600135
bpxsy4_systolic__blood_pres_4th_rdg_mm_hg,94.071702
bpxdi4_diastolic__blood_pres_4th_rdg_mm_hg,94.071702
diq175h_high_blood_pressure,97.240953


In [35]:
drop_columns = list(df_high.index)
df.drop(drop_columns, axis=1, inplace=True)

In [36]:
df.shape

(17796, 50)

### 4. Checking data types

In [37]:
# data type
print(df.dtypes.unique()) # all are float64 

[dtype('float64') dtype('int64') dtype('O')]


In [38]:
df.dtypes[df.dtypes == np.object]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


salt_usage       object
insulin_usage    object
dtype: object

In [39]:
## Columns with string values
col_types = list(filter(lambda i: df[i].dtype==np.object , df.columns))

for column in col_types:
    print("Column name :",str(column))
    print("Unique values :",str(df[column].unique()))
    print('\n')

Column name : salt_usage
Unique values : [0 1]


Column name : insulin_usage
Unique values : ['None' 'More than one year' '2 Month' '3-4 Months' '5-12 Months']




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [40]:
df = df.drop(columns=['bpxsy1_systolic__blood_pres_1st_rdg_mm_hg'
                      ,'bpxdi1_diastolic__blood_pres_1st_rdg_mm_hg'
                      ,'bpxsy2_systolic__blood_pres_2nd_rdg_mm_hg'
                      ,'bpxdi2_diastolic__blood_pres_2nd_rdg_mm_hg'
                      ,'bpxsy3_systolic__blood_pres_3rd_rdg_mm_hg'
                      ,'bpxdi3_diastolic__blood_pres_3rd_rdg_mm_hg'
                      ,'BPX_SY'
                      ,'BPX_DI'
                      ,'BPX'
                      ,'flag'
                      ,'seqn_respondent_sequence_number'
                      ,'alq141u_days_drink_alcohol_per_wk_mo_yr'
                      ,'alq151_ever_have_4_5_or_more_drinks_every_day'
                      ,'bpq020_ever_told_you_had_high_blood_pressure'
                     ])

### 5. Renaming columns and mapping

In [41]:
df = df.rename(columns={
    'seqn_respondent_sequence_number': 'id',
    'bpxpuls_pulse_regular_or_irregular':'pulse_regularity',
    'riagendr_gender':'gender',
    'ridageyr_age_in_years_at_screening':'age',
    'pad680_minutes_sedentary_activity':'mins_sedentary_activity',
    'pad630_minutes_moderate_intensity_work':'mins_moderate_intensity_activity',
    'paq625_number_of_days_moderate_work':'number_of_days_moderate_intensity_activity',
    'ridreth3_racehispanic_origin_w_nh_asian':'race',
    'dmdeduc2_education_level':'education',
    'dmdmartl_marital_status':'marital_status',
    'smq020_smoked_at_least_100_cigarettes_in_life':'smoke',
    'dr1tpota_potassium_mg':'potassium', 
    'dr1tcaff_caffeine_mg':'caffeine',
    'dr1tsodi_sodium_mg':'sodium',
    'dmqmiliz_served_active_duty_in_us_armed_forces':'active_duty',
    'dbq095z_type_of_table_salt_used':'salt_type',
    'alq111_ever_had_a_drink_of_any_kind_of_alcohol':'alcohol',
    'alq142_number_days_have_4_or_5_drinkspast_12_mos':'alcohol_usage',
    'drd370g_haddock_eaten_during_past_30_days':'haddock',
    'drd370h_mackerel_eaten_during_past_30_days':'mackerel',
    'drd370i_perch_eaten_during_past_30_days':'perch',
    'drd370l_porgy_eaten_during_past_30_days':'porgy',
    'drd370o_sea_bass_eaten_during_past_30_days':'sea_bass',
    'drd370r_trout_eaten_during_past_30_days':'trout',
    'diq050_taking_insulin_now': 'insulin',
    'did060_how_long_taking_insulin': 'how_long_taking_insulin',
    'diq060u_unit_of_measure_month_year' : 'insulin_unit_month_year',
    'diq010_doctor_told_you_have_diabetes': 'diabetes',
    'diq060_ever_told_you_have_prediabetes': 'prediabetes',
    'dbd100_how_often_add_salt_to_food_at_table':'salt_add_at_table',
    'drqsprep_salt_used_in_preparation':'salt_add_in_preparation',
    'dr1tchol_cholesterol_mg': 'cholesterol',
    'lbxin_insulin_uu_ml': 'insulin_uu_ml'
})

### 6. Creating dataset for modeling

In [42]:
# BMI convertion for modelling
df['bmi_range'] = None
df.loc[df['BMI'] < 18.5, 'bmi_range']= 1
df.loc[(df['BMI'] >= 18.5) & (df['BMI'] <= 24.9) ,'bmi_range']= 2 # Normal
df.loc[(df['BMI'] >= 25) & (df['BMI'] <= 29.9),'bmi_range']= 3 # Overweight
df.loc[(df['BMI'] > 30),'bmi_range']= 4 # Obese

df.bmi_range.value_counts()

4    6150
3    5595
2    5063
1     666
Name: bmi_range, dtype: int64

In [43]:
df_model = df.copy()

# droping rows with null BMI
df_model = df_model.dropna(subset=['bmi_range'])
df_model['bmi_range'] = df_model['bmi_range'].astype('int')
# Droping insulin usage column from the modeling dataset
df_model = df_model.drop(columns=['insulin_usage'])

In [44]:
# Using quantile for mg related columns

#df_model['caffeine'].dropna().quantile([0.0, .25, .75, .90])
category = pd.cut(df_model.caffeine, bins=[0,11,200,350,5500],labels=[1,2,3,5])
df_model.insert(0,'caffeine_q',category)
df_model['caffeine_q'] = df_model['caffeine_q'].astype('float')

#df_model['potassium'].dropna().quantile([0.0, .25, .75, .90])
category = pd.cut(df_model.potassium, bins=[0,1744,3278,4218,20000],labels=[1,2,3,5])
df_model.insert(0,'potassium_q',category)
df_model['potassium_q'] = df_model['potassium_q'].astype('float')

#df_model['sodium'].dropna().quantile([0.0, .25, .75, .90])
category = pd.cut(df_model.sodium, bins=[0,2256,4384,5859,26000],labels=[1,2,3,5])
df_model.insert(0,'sodium_q',category)
df_model['sodium_q'] = df_model['sodium_q'].astype('float')


df_model['mins_sedentary_activity'].dropna().quantile([0.0, .25, .75, .90])
category = pd.cut(df_model.mins_sedentary_activity,bins=[0,180,480,600,1440],labels=[1,2,3,5])
df_model.insert(0,'sedentary_q',category)

Converting yes, no column to 0, 1

In [45]:
convert_columns = ['gender', 'active_duty', 'pulse_regularity','smoke','insulin','prediabetes','alcohol','sleep_disorder']
for column in convert_columns:
    df_model[column] = df_model[column].map({1: 0, 2: 1, 7:9})

In [46]:
joblib.dump(df_model, '/dsa/groups/capstonesp2022/on-campus/group_1/data/clean_dataset.pkl')

['/dsa/groups/capstonesp2022/on-campus/group_1/data/clean_dataset.pkl']

### 7.Preparing dataset for Visualization

In [47]:
null=100*(df.isnull().sum())/(df.shape[0])
df_null=pd.DataFrame({'percentage':null})
df_high=df_null[df_null['percentage']>0]
display(df_high.sort_values(by=['percentage']))

,percentage
smoke,0.005619
insulin,0.005619
mins_sedentary_activity,0.162958
sleep_hour,0.320297
BMI,1.022702
bmi_range,1.809395
pulse_regularity,4.725781
sodium,10.170825
potassium,10.170825
caffeine,10.170825


In [48]:
df['salt_add_at_table'] = df['salt_add_at_table'].map({1:'Rarely',
                                                         2:'Occasionally',
                                                         3:'Very Often', 
                                                         0:'Unkown'})
df.salt_add_at_table.value_counts()

Unkown          6820
Rarely          5262
Occasionally    3293
Very Often      2421
Name: salt_add_at_table, dtype: int64

In [49]:
df['salt_add_in_preparation'] = df['salt_add_in_preparation'].map({1:'Never',
                             2:'Rarely',
                             3:'Occasionally', 
                             4:'Very Often', 
                             0:'Unkown'})
df.salt_add_in_preparation.value_counts()

Very Often      6871
Occasionally    4912
Rarely          2748
Unkown          2030
Never           1235
Name: salt_add_in_preparation, dtype: int64

In [50]:
df['age_group'] = None
df.loc[df['age'] <= 40 ,'age_group']= '20 - 40'
df.loc[(df['age'] > 40) & (df['age'] <= 60),'age_group']= '40 - 60'
df.loc[(df['age'] > 60),'age_group']= '60+'
df.age_group.value_counts()

20 - 40    7104
40 - 60    6249
60+        4443
Name: age_group, dtype: int64

In [51]:
df['active_duty'] = df['active_duty'].map({1:'Yes',2:'No'})
df.active_duty.value_counts()

No     16470
Yes     1324
Name: active_duty, dtype: int64

In [52]:
df['pulse_regularity'] = df['pulse_regularity'].map({1:'Regular',2:'Irregular'})
df.pulse_regularity.value_counts()

Regular      16602
Irregular      353
Name: pulse_regularity, dtype: int64

In [53]:
df['bmi_range'] = None
df.loc[df['BMI'] < 18.5, 'bmi_range']= 'Underweight'
df.loc[(df['BMI'] >= 18.5) & (df['BMI'] <= 24.9) ,'bmi_range']= 'Normal' # Normal
df.loc[(df['BMI'] >= 25) & (df['BMI'] <= 29.9),'bmi_range']= 'Overweight' # Overweight
df.loc[(df['BMI'] > 30),'bmi_range']= 'Obese' # Obese
df.bmi_range.value_counts()

Obese          6150
Overweight     5595
Normal         5063
Underweight     666
Name: bmi_range, dtype: int64

In [54]:
df['alcohol_usage'] = df['alcohol_usage'].map({  1: '1. Every day'
                                               , 2: '2. Nearly every day'
                                               , 3: '3. 3 to 4 times a week'
                                               , 4: '4. 2 times a week'
                                               , 5: '5. Once a week'
                                               , 6: '6. 2 to 3 times a month'
                                               , 7: '7. Once a month'
                                               , 8: '8. 7 to 11 times in the last year'
                                               , 9: '9. 3 to 6 times in the last year'
                                               , 10:'10. 1 to 2 times in the last year'
                                               , 0: '11. Never last year'
                                               , 99:"12. Don't know"
                                              })
df.alcohol_usage.value_counts()

11. Never last year                  5615
12. Don't know                       5465
10. 1 to 2 times in the last year    1565
9. 3 to 6 times in the last year     1194
6. 2 to 3 times a month               870
5. Once a week                        746
7. Once a month                       710
4. 2 times a week                     547
8. 7 to 11 times in the last year     406
3. 3 to 4 times a week                348
1. Every day                          198
2. Nearly every day                   132
Name: alcohol_usage, dtype: int64

In [55]:
df.alcohol_usage.value_counts()

11. Never last year                  5615
12. Don't know                       5465
10. 1 to 2 times in the last year    1565
9. 3 to 6 times in the last year     1194
6. 2 to 3 times a month               870
5. Once a week                        746
7. Once a month                       710
4. 2 times a week                     547
8. 7 to 11 times in the last year     406
3. 3 to 4 times a week                348
1. Every day                          198
2. Nearly every day                   132
Name: alcohol_usage, dtype: int64

In [56]:
df['smoke'] = df['smoke'].map({1:'Yes', 2:'No'})
df.smoke.value_counts()

No     10143
Yes     7646
Name: smoke, dtype: int64

In [57]:
df['gender'].replace(1.0,'Male', inplace=True)
df['gender'].replace(2.0,'Female', inplace=True)
df.gender.value_counts()

Female    9195
Male      8601
Name: gender, dtype: int64

In [58]:
df['race'] = df['race'].map({1:'Mexican American',
                             2:'Other Hispanic',
                             3:'White', 
                             4:'Black', 
                             5:'Multi-Racial',
                             6:'Asian'})
df.race.value_counts()

White               6234
Black               3907
Mexican American    2987
Other Hispanic      1995
Asian               1930
Multi-Racial         743
Name: race, dtype: int64

In [59]:
df['alcohol'].replace(1.0,'Yes', inplace=True)
df['alcohol'].replace(2.0,'No', inplace=True)
df['alcohol'].replace(9.0,"Don't know", inplace=True)

df.alcohol.value_counts()

Yes           9625
No            5615
Don't know    2556
Name: alcohol, dtype: int64

In [60]:
df['sleep_disorder'].replace(1.0,'Yes', inplace=True)
df['sleep_disorder'].replace(2.0,'No', inplace=True)
df['sleep_disorder'].replace(9.0,"Don't know", inplace=True)

df.sleep_disorder.value_counts()

No            10164
Yes             963
Don't know       12
Name: sleep_disorder, dtype: int64

In [61]:
df['insulin'].replace(1.0,'Yes', inplace=True)
df['insulin'].replace(2.0,'No', inplace=True)
df['insulin'].replace(9.0,"Don't know", inplace=True)

df.insulin.value_counts()

No     17100
Yes      695
Name: insulin, dtype: int64

In [62]:
df['diabetes'].replace(1.0,'Yes', inplace=True)
df['diabetes'].replace(2.0,'No', inplace=True)
df['diabetes'].replace(3.0,'Borderline', inplace=True)
df['diabetes'].replace(9.0, "Don't know", inplace=True)
df.diabetes.value_counts()

No            15036
Yes            2349
Borderline      402
Don't know        9
Name: diabetes, dtype: int64

In [63]:
df['prediabetes'].replace(1.0,'Yes', inplace=True)
df['prediabetes'].replace(2.0,'No', inplace=True)
df['prediabetes'].replace(9.0, "Don't know", inplace=True)
df.prediabetes.value_counts()

No            13962
Yes            1059
Don't know       24
Name: prediabetes, dtype: int64

In [64]:
df['insulin_unit_month_year'].replace(0.0,'None', inplace=True)
df['insulin_unit_month_year'].replace(1.0,'Months', inplace=True)
df['insulin_unit_month_year'].replace(2.0,'Year', inplace=True)
df.insulin_unit_month_year.value_counts()

None      17118
Year        592
Months       86
Name: insulin_unit_month_year, dtype: int64

In [65]:
df['HBP'] = df['HBP'].map({0:'No',1:'Yes'})
df.HBP.value_counts()

No     8976
Yes    8820
Name: HBP, dtype: int64

In [66]:
joblib.dump(df, '/dsa/groups/capstonesp2022/on-campus/group_1/data/visualization_dataset.pkl')

['/dsa/groups/capstonesp2022/on-campus/group_1/data/visualization_dataset.pkl']

In [67]:
#df.to_csv('dashboard.csv')